In [1]:
print('yaya')

yaya


Concepts:
- Lesson 1
    - Individuals and Categorical/Numerical Variables/Data
    - Pictographs
        - Could import a bunch of emojis and generate graphs that way? But how would I insert that into a chart... I could do it via a print statement
        - See this link for how to implement in matplotlib: https://www.geeksforgeeks.org/emojis-as-markers-in-matplotlib/
    - Bar Charts
        - Could use a variety of packages just for "fun"
    - Mean, Median, Mode, Midrange, Range (Measures of Central Tendency)
    - Pie Charts
- Lesson 2
    - Venn Diagram
    - 2 Way Tables + 2 Way Relative Frequency Tables


Goals:
- I want to randomly generate data, so it could be good to generate classical datasets (iris) or we could find datasets on Kaggle
- It will be good to provide an explanation of each of these
- Do I want this to act as if I am preparing for someone else to study this? Regardless, it will be good to include explanations in markdown cells
- I will want to use my own generated functions to test indepence for 2 way frequency table questions, I could have a separate file that I use where I implement my own functions to hide the syntax and clean things up
- I think it will get good to have the opportunity to check your answers --> this will force me to ensure I understand how to calculate these things.